# Some preliminary checks

In [ ]:
import torch
import os

os.environ["RAY_DEDUP_LOGS"] = "0"

import ray 
print("Ray version :", ray.__version__)

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)

print("MPS Available:", torch.backends.mps.is_available())
print("onnx Available:", torch.onnx.is_onnxrt_backend_supported())
torch._dynamo.list_backends()

### Important : Number of CPUs and GPUs available

In [ ]:
import psutil

# print number of gpus / CPUs
print("Number of CPUs: ", psutil.cpu_count())

num_cpus = 16
num_gpus = 1
num_learner = 1

assert num_cpus <= psutil.cpu_count()

# Environement and algorithm configuration

Some of the commented lines are preparation work to use a futur feature of RLLib

Note: In multi-agent environments, `rollout_fragment_lenght` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.


In [ ]:
from ray.rllib.policy.policy import PolicySpec
#from ray.rllib.core.rl_module.rl_module import SingleAgentRLModuleSpec
#from ray.rllib.core.rl_module.marl_module import MultiAgentRLModuleSpec

from ray.tune.registry import get_trainable_cls

from particle_2d_env import Particle2dEnvironment
from particle_2d_env import MetricsCallbacks
from config import run_config

ALGO = "PPO"        
FRAMEWORK= "torch" # "tf2" or "torch"
env = Particle2dEnvironment(run_config["env"])

config = (
    get_trainable_cls(ALGO).get_default_config()
    .environment(Particle2dEnvironment, env_config=run_config["env"])
    .framework(
        FRAMEWORK,
    )
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=False,
    )
    .callbacks(MetricsCallbacks)
    .resources(
        # learner workers when using learner api - doesn't work on arm (mac) yet
        num_learner_workers=num_learner,
        num_gpus_per_learner_worker=num_gpus/num_learner, # always 1 for PPO
    )
    # Specify the learner's hyperparameters.
    .training(
        num_sgd_iter=5,          
        sgd_minibatch_size=512,             # the batch size (only for PPO)
        #minibatch_size = 256,                # the batch size (only for IMPALA)
        train_batch_size=524288,             # the number of step collected
        model={
            "fcnet_hiddens": [128, 128, 128], 
            "use_attention": True,
            #"use_lstm": False,
            #"max_seq_len": 5,
            #"lstm_cell_size": 16,
        },
        #lr=tune.grid_search([0.01, 0.001, 0.0001])
    )
    .multi_agent(
        policies= {
            "prey": PolicySpec(
                policy_class=None,  # infer automatically from Algorithm
                observation_space=env.observation_space[0],  # if None infer automatically from env
                action_space=env.action_space[0],  # if None infer automatically from env
                config={"gamma": 0.85},  # use main config plus <- this override here
            ),
            "predator": PolicySpec(
                policy_class=None,
                observation_space=env.observation_space[0],
                action_space=env.action_space[0],
                config={"gamma": 0.85},
            ),
        },
        policy_mapping_fn = lambda id, *arg, **karg: "prey" if env.agents[id].agent_type == 0 else "predator",
        policies_to_train=["prey", "predator"],
        count_steps_by="agent_steps",
    )
    .env_runners(
        rollout_fragment_length="auto", #"auto" for PPO explained here : https://docs.ray.io/en/latest/rllib/rllib-sample-collection.html
        batch_mode= 'truncate_episodes',
        num_env_runners=num_cpus-num_learner*0-1, # need 2 for IMPALA, 1 for PPO
        num_envs_per_env_runner=2,
    )
    .checkpointing(export_native_model_files=True)
)


# Training

In [ ]:
from ray.rllib.policy.policy import Policy
from ray.rllib.algorithms.callbacks import DefaultCallbacks

path_to_checkpoint = None #os.getcwd() + "/ray_results" + "PPO_2024-05-18_00-08-19/PPO_Particle2dEnvironment_bb60c_00000_0_2024-05-18_00-08-19/checkpoint_000001"

def restore_weights(path, policy_type):
    checkpoint_path = os.path.join(path, f"policies/{policy_type}")
    restored_policy = Policy.from_checkpoint(checkpoint_path)
    return restored_policy.get_weights()

if path_to_checkpoint is not None: 
    class RestoreWeightsCallback(DefaultCallbacks):
        def on_algorithm_init(self, *, algorithm: "Algorithm", **kwargs) -> None:
            algorithm.set_weights({"predator": restore_weights(path_to_checkpoint, "predator")})
            algorithm.set_weights({"prey": restore_weights(path_to_checkpoint, "prey")})

    config.callbacks(RestoreWeightsCallback)



## Launch training

In [ ]:
from ray import train, tune
from ray.tune import Tuner
from ray.air.integrations.wandb import WandbLoggerCallback
from ray.rllib.utils.test_utils import check_learning_achieved
import os

ray.init(
    num_cpus=num_cpus, 
    num_gpus=num_gpus
)

# Stop criterium
stop = {  
    "training_iteration": 1500,
    #"timesteps_total": 200000000,
}

# Read the API key from the file to use Wanddb
with open('wandb_api_key.txt', 'r') as file:
    api_key = file.read().strip()
callbacks = [
    WandbLoggerCallback(                   
        project="marl-rllib", 
        group=ALGO,
        api_key=api_key,
        log_config=True,
        upload_checkpoints=True
    ), 
]

# When to save the models 
checkpoint_config = train.CheckpointConfig(         
    checkpoint_at_end=True,
    checkpoint_frequency=10,
)

# Where to save 
# absolute path + ray_results directory
storage_path=os.getcwd() + "/ray_results"

if path_to_checkpoint is None : 
    tuner = tune.Tuner(
        ALGO,                                                 # Defined before
        param_space=config,                                   # Defined before
        run_config=train.RunConfig(    
            storage_path=storage_path,
            stop=stop,
            verbose=3,
            callbacks=callbacks,
            checkpoint_config=checkpoint_config,
        ),
    )
    # Run the experiment 
    results = tuner.fit()

# If we instantiate previously trained neural network
else: 
    callbacks.append(RestoreWeightsCallback)

    results = tune.run(
        ALGO,
        config=config.to_dict(),
        storage_path=storage_path,
        stop=stop,
        verbose=3,
        callbacks=callbacks,
        checkpoint_config=checkpoint_config,
    )


ray.shutdown()
